In [56]:
import os, sys
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
sys.path.append('./s2spy')
import preprocessing
from s2spy import RGDR, time
import func_SPI, utils, h5py
path_obs_data = '/data/volume_2/observational/'
filename = 'chrips_precip_1981-2021.nc'
path_tp = os.path.join(path_obs_data, 'raw', filename)

In [115]:
# load intervals and target periods from EC46 initialization dates
dfs = pd.read_hdf('/data/volume_2/subseasonal/ecmwf/aggregated/aggregation_timestamps.h5').sort_index()
df = dfs[['aggregation_start_inclusive', 'aggregation_end_inclusive']].copy()
sel_months = [10, 11, 12]
mon_start_mask = [True if m in sel_months else False for m in df.set_index('aggregation_start_inclusive').index.month]
mon_end_mask = [True if m in sel_months else False for m in df.set_index('aggregation_end_inclusive').index.month]
mon_mask = np.logical_and(mon_start_mask, mon_end_mask)
df.iloc[360:380]

,aggregation_start_inclusive,aggregation_end_inclusive
init_date,,
2006-10-22,2006-11-06,2006-12-03
2006-10-29,2006-11-13,2006-12-10
2006-11-05,2006-11-20,2006-12-17
2006-11-12,2006-11-27,2006-12-24
2006-11-19,2006-12-04,2006-12-31
2006-11-26,2006-12-11,2007-01-07
2006-12-03,2006-12-18,2007-01-14
2006-12-10,2006-12-25,2007-01-21
2006-12-17,2007-01-01,2007-01-28


In [118]:
tp = xr.open_dataarray(path_tp)
# select region
tp = tp.sel(latitude=slice(-5,8), longitude=slice(38,53))
# spatial mean
tp_sm = tp.mean(dim=('latitude', 'longitude'))
tp_sm

<xarray.DataArray 'precip' (time: 14975)>
array([0.0000000e+00, 4.8481959e-01, 2.6115529e-02, ..., 4.2302564e-01,
       4.3223627e-02, 4.4919315e-04], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2021-12-31

In [119]:
# 28-day rolling mean
tp_rm = tp_sm.rolling({'time':28}, min_periods=1, center=True).mean()

In [120]:
quantiles = tp_rm.groupby(tp_rm.time.dt.dayofyear).quantile(q=.33, dim='time', skipna=True)
quantiles.name = '0.33'
quantiles

<xarray.DataArray '0.33' (dayofyear: 366)>
array([0.15870125, 0.16130118, 0.1602353 , 0.16664413, 0.163122  ,
       0.16224493, 0.15494163, 0.15962094, 0.14233505, 0.12464583,
       0.10831681, 0.11645305, 0.11834736, 0.10933021, 0.10373813,
       0.11316288, 0.10858094, 0.10321271, 0.0965468 , 0.09643591,
       0.09469874, 0.09488363, 0.09539632, 0.08522961, 0.08229516,
       0.08540759, 0.08443118, 0.09080722, 0.0910161 , 0.09264203,
       0.09127538, 0.10215352, 0.12199633, 0.12825343, 0.12834638,
       0.12286096, 0.12878813, 0.13081736, 0.13849336, 0.14181847,
       0.13599763, 0.13321972, 0.15216017, 0.16327419, 0.16763127,
       0.16837562, 0.18364023, 0.18025673, 0.19211178, 0.1749154 ,
       0.1952204 , 0.18759964, 0.19418497, 0.1846394 , 0.1771765 ,
       0.18591073, 0.1683515 , 0.17942898, 0.2063292 , 0.2088068 ,
       0.19854017, 0.19461265, 0.19733666, 0.19420467, 0.21126394,
       0.22931904, 0.24707928, 0.27681366, 0.31145852, 0.34722347,
       0.32579687, 0.33327683, 0.40678015, 0.44525903, 0.47318448,
       0.53771191, 0.59840527, 0.59384928, 0.61831367, 0.67309701,
       0.68897771, 0.76185713, 0.90110389, 0.90284725, 0.95237861,
       1.04204922, 1.14272294, 1.33152194, 1.34221053, 1.50426028,
       1.5352211 , 1.74471116, 1.771931  , 1.96447091, 2.09520025,
       2.22356415, 2.20007319, 2.33333511, 2.36356516, 2.34478159,
...
       0.80767161, 0.87712631, 0.90396674, 0.92433883, 1.00371456,
       1.10407178, 1.07486989, 1.07822711, 1.15091264, 1.16999695,
       1.23134155, 1.23602402, 1.25558565, 1.23115544, 1.2977155 ,
       1.36725662, 1.40466826, 1.45097582, 1.57584016, 1.54432416,
       1.55185077, 1.64442992, 1.66610076, 1.6806844 , 1.72784944,
       1.75894299, 1.73547573, 1.6383554 , 1.62475219, 1.66412065,
       1.69092238, 1.697082  , 1.68257935, 1.69508479, 1.58915248,
       1.47411525, 1.55390751, 1.59497371, 1.57130144, 1.58784883,
       1.6409445 , 1.65256181, 1.69594367, 1.61138163, 1.66021631,
       1.59894221, 1.49985571, 1.43151619, 1.48564122, 1.42446039,
       1.36931753, 1.32733228, 1.30657423, 1.23376882, 1.21855569,
       1.18436162, 1.15656106, 1.10580482, 1.09070883, 1.08825564,
       1.10621576, 1.17243562, 1.11237993, 0.93646269, 0.92088175,
       0.82374479, 0.88861455, 0.87208858, 0.8623396 , 0.83695356,
       0.82476575, 0.8120901 , 0.74863809, 0.7269014 , 0.68181416,
       0.66108757, 0.63343939, 0.63153273, 0.6315021 , 0.59748528,
       0.52909535, 0.50944179, 0.48686999, 0.46819514, 0.45725304,
       0.40976018, 0.31608163, 0.27174468, 0.22954546, 0.21373285,
       0.2206379 , 0.21358332, 0.20853708, 0.20278758, 0.18775887,
       0.15705588])
Coordinates:
    quantile   float64 0.33
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366

In [121]:
tp_tercile = (tp_rm.groupby(tp_rm.time.dt.dayofyear) < quantiles).astype(int).drop('dayofyear').drop('quantile')

In [123]:
ds = tp_tercile.to_dataset(name='binary')
ds['tp_28d_rm'] = tp_rm
ds['quantile'] = quantiles
ds['spatial_mean_raw'] = tp_sm

In [124]:
ds.to_netcdf(os.path.join(path_obs_data, 'chrips_1981-2021_target.nc'))

Note that the rolling mean is centered.

In [114]:
# select center of 28-day rolling mean
df['center'] = df[['aggregation_start_inclusive']] + pd.Timedelta('14d')
tp_rm_s = tp_rm.sel(time=pd.to_datetime(df['center'].values)) 
# label alignment left
tp_rm_s['time'] = pd.to_datetime(df[['aggregation_start_inclusive']].values)
tp_rm_s

<xarray.DataArray 'precip' (time: 1113)>
array([0.10109671, 0.06325012, 0.06289413, ..., 0.15035173, 0.08285904,
       0.08602343], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-17 2000-01-24 ... 2021-01-15

In [ ]:
# later

In [ ]:
aggr = 1
path_spi = filename.split('_')[0] + f'_SPI_{aggr}_' + '_'.join(filename.split('_')[2:])
output = func_SPI.calc_SPI_from_daily(tp.to_array(), aggr)

In [ ]:
output.to_netcdf(os.path.join(path_obs_data, 'preprocessed', path_spi))

In [ ]:
latitude = 0 ; longitude=30
ts_raw = tp.to_array().sel(latitude=latitude, longitude=longitude, method='nearest')
ts_stn = output.sel(latitude=latitude, longitude=longitude, method='nearest')
# ts_pack = SMI_package.sel(latitude=40, longitude=240)

fig = plt.figure(figsize=(20,10) )

# plot observed versus corresponding Gamma probability
ax1 = plt.subplot(1, 2, 1)
vals = ax1.plot(ts_raw, '.k');
# ax1.set_ylabel('Cumulative probability (from Gamma distribution)')
# ax1.set_xlabel('Aggregated Precipitation [mm] - Original values')
# plot transformed standard normal from Gamma probability
ax1 = plt.subplot(1, 2, 2)
vals = ax1.plot(ts_stn, '.k');
# ax1.plot(ts_pack, '.r')
# ax1.set_ylabel('Cumulative probability (from Gamma distribution)')
ax1.set_xlabel('SPI - Gamma prob. transformed to standard normal ')